In [9]:
from skimage.io import imread
from magicgui import magicgui, type_map
import napari
from napari.utils import nbscreenshot

from my_module import preprocess_image, segment_image, subtract_bg_and_denoise, extract_features
from napari.types import ImageData, LabelsData, LayerDataTuple

In [2]:
image = imread("../../data/blobs.tif")

In [3]:
viewer = napari.Viewer()
viewer.add_image(image)

INFO:xmlschema:Resource 'XMLSchema.xsd' is already loaded


<Image layer 'image' at 0x2940da25d30>

In [4]:
preprocess_widget = magicgui(preprocess_image)

In [5]:
viewer.window.add_dock_widget(preprocess_widget)

In [6]:
segment_widget = magicgui(segment_image)
viewer.window.add_dock_widget(segment_widget)

In [7]:
subtract_bg_and_denoise_widget = magicgui(subtract_bg_and_denoise)
viewer.window.add_dock_widget(subtract_bg_and_denoise_widget)

In [10]:
extract_features_widget = magicgui(extract_features)
viewer.window.add_dock_widget(extract_features_widget)